<a href="https://colab.research.google.com/github/greymouse1/ss_lab1/blob/main/ss_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import numpy for matrix operations
import numpy as np
# import math for advanced operations
import math

# import modules for handling external files
import requests
from io import StringIO

Task #1

In [2]:
# define known values (everything is in meters and dgm)

# GRS80 parameters
a = 6378137
f = 1/298.257222101


# semi-minor axis
b = a*(1-f)

# ellipsoid height
h = 85

# eccentricity
e_sq = ((a**2)-(b**2))/(a**2)

# Lat
lat_d = 59
lat_m = 20
lat_s = 59

# Lon
lon_d = 18
lon_m = 4
lon_s = 10

# define functions for conversion from dgm to radians

def dgm_rad (d,m,s) :
  decimal_degrees = d + m/60 + s/3600
  print(f"Decimal degrees are {decimal_degrees}")
  degrees_radians = decimal_degrees * (math.pi / 180)
  return degrees_radians

# Lat in rads
lat_r = dgm_rad(lat_d,lat_m,lat_s)

# Long in rads
lon_r = dgm_rad(lon_d,lon_m,lon_s)

N = a / math.sqrt(1 - e_sq * math.sin(lat_r)**2)

# define functions for conversion of geodetic to cartesian coordinates
X = (N+h)*(math.cos(lat_r)*math.cos(lon_r))
Y = (N+h)*(math.cos(lat_r)*math.sin(lon_r))
Z = (N*(1-e_sq)+h)*math.sin(lat_r)

print(f"X coordinate is {X}")
print(f"Y coordinate is {Y}")
print(f"Z coordinate is {Z}")

def geocentric_geodetic(X,Y,Z,e_sq):

  # define auxiliary values
  p = math.sqrt(X**2 + Y**2)

  teta = math.atan(Z / (p*math.sqrt(1 - e_sq)))

  # define functions for conversion from cartesian to geodetic

  lon = math.degrees(math.atan(Y/X))
  lat = math.degrees(math.atan((Z + ((a*e_sq)/math.sqrt(1-e_sq))*math.sin(teta)**3) / (p - a*e_sq*math.cos(teta)**3) ))
  h_reverse = (p / math.cos(math.radians(lat))) - N

  return(lon,lat,h_reverse)

calc_values = geocentric_geodetic(X,Y,Z,e_sq)
print(calc_values)
#print(f"Longitude is {lon}")
#print(f"Latitude is {lat}")
#print(f"h is {h_reverse}")

Decimal degrees are 59.349722222222226
Decimal degrees are 18.069444444444443
X coordinate is 3098917.2417289796
Y coordinate is 1011053.4113269701
Z coordinate is 5463972.352679947
(18.069444444444443, 59.34972222222222, 84.99999999906868)


Task #2

In [15]:
# Satellite positions file path
external_link = 'https://raw.githubusercontent.com/greymouse1/ss_lab1/main/satellites.txt'

# Download data from the external link
response = requests.get(external_link)
data = np.genfromtxt(StringIO(response.text), delimiter=' ')

# Create a dictionary with satellite names as keys and coordinate vectors as values
sat_dict = {}

for row in data:
    sat_name = row[0] # Convert bytes to string for the point name
    sat_coordinates = np.array(row[1:4], dtype=float)  # Create a NumPy array for coordinates
    sat_dict[str(int(sat_name))] = 1000*sat_coordinates  # Add to the dictionary

# Create rotation matrix Rl
R1 = np.array([[1, 0, 0],
               [0,math.cos(90 * (math.pi / 180)-lat_r) ,math.sin(90 * (math.pi / 180)-lat_r) ],
               [0, -1*math.sin(90 * (math.pi / 180)-lat_r),math.cos(90 * (math.pi / 180)-lat_r) ]])

R3 = np.array([[math.cos(lon_r+90 * (math.pi / 180)) ,math.sin(lon_r+90 * (math.pi / 180)), 0],
               [-1*math.sin(lon_r+90 * (math.pi / 180)),math.cos(lon_r+90 * (math.pi / 180)) , 0],
               [0, 0, 1]])

Rl = np.dot(R1,R3)

print("Rl matrix is: ", "\n" , Rl)

# Convert all satellite coordinates to ENU coordinates

sat_enu = {}

origin_vector = np.array([X, Y, Z])

for sat in sat_dict:
  sat_enu[sat] = np.dot(Rl,sat_dict[sat] - origin_vector)

# Create function for calculating azimuth

def azimuth(e,n):
  azimuth_rad = math.atan2(e, n)
  azimuth_deg = math.degrees(azimuth_rad)

  # Ensure azimuth is in the range [0, 360)
  azimuth_deg = (azimuth_deg + 360) % 360

  return azimuth_deg

# Dictionary containing only satellites above horizon

enu = {}

for sat in sat_enu:
  e = sat_enu[sat][0]
  n = sat_enu[sat][1]

  # height
  u = sat_enu[sat][2]
  # slant distance
  s = math.sqrt(n**2+e**2+u**2)

  # zenith
  z = math.degrees(math.acos(u / s))

  # elevation
  ele = 90 - z

  if ele > 0:
    # pack all values
    values = [azimuth(e,n),ele,s]
    enu[sat] = values

print("Satellites which are visible by observer and their elevation :")

for key in enu:
  print(key,",",enu[key][1])




Rl matrix is:  
 [[-0.31016948  0.95068128  0.        ]
 [-0.81786636 -0.26683725  0.50979653]
 [ 0.48465402  0.15812333  0.86029501]]
Satellites which are visible by observer and their elevation :
1 , 31.046238428766173
3 , 13.273751623676517
7 , 1.0947928824200375
9 , 13.83736876483212
11 , 57.43066537619199
14 , 47.52233856645591
19 , 42.870700858933155
20 , 11.05266718535735
22 , 32.02878579501622
28 , 22.508841540234556
31 , 20.299680386993387


# New section